# Consignes

Intégrer le fichier USvideos.csv qui représente un ensemble de 8000 vidéos Youtube. 

Merger le fichier US_category_id.json pour récupérer le nom des catégories. Il conviendra de bien spécifier l'ID du document.


# Questions 
- 1) Récupérer toutes les vidéos de la chaîne Apple.
- 2) Compter le nombre de catégories différentes
- 3) Si vous ne l'avez pas déjà fait, découper les tags en listes et mettre à jour les tags de chacun des documents avec une requête update.
- 4) Récupérer les vidéos les plus vues.
- 5) Compter le nombre moyen de vues en fonction de la catégorie.
- 6) Récupérer les chaines Youtube avec la plus grande moyenne de likes.

In [16]:
import pandas as pd
import pymongo

In [17]:
client = pymongo.MongoClient('mongodb://localhost:27017/')
database = client['exercices']
collection = database['youtube']

In [18]:
df_youtube = pd.read_csv("./data/USvideos.csv",parse_dates=['date'])
df_youtube.head()

,video_id,title,channel_title,category_id,tags,views,likes,dislikes,comment_total,thumbnail_link,date
0,XpVt6Z1Gjjo,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,Logan Paul Vlogs,24,logan paul vlog|logan paul|logan|paul|olympics...,4394029,320053,5931,46245,https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg,13.09
1,K4wEI5zhHB0,iPhone X — Introducing iPhone X — Apple,Apple,28,Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lig...,7860119,185853,26679,0,https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg,13.09
2,cLdxuaxaQwc,My Response,PewDiePie,22,[none],5845909,576597,39774,170708,https://i.ytimg.com/vi/cLdxuaxaQwc/default.jpg,13.09
3,WYYvHb03Eog,Apple iPhone X first look,The Verge,28,apple iphone x hands on|Apple iPhone X|iPhone ...,2642103,24975,4542,12829,https://i.ytimg.com/vi/WYYvHb03Eog/default.jpg,13.09
4,sjlHnJvXdQs,iPhone X (parody),jacksfilms,23,jacksfilms|parody|parodies|iphone|iphone x|iph...,1168130,96666,568,6666,https://i.ytimg.com/vi/sjlHnJvXdQs/default.jpg,13.09


## Question 0

### Netoyer les données

In [19]:
df_youtube.count()

video_id          7992
title             7992
channel_title     7992
category_id       7992
tags              7992
views             7992
likes             7992
dislikes          7992
comment_total     7992
thumbnail_link    7992
date              7992
dtype: int64

In [20]:
df_youtube.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7992 entries, 0 to 7991
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   video_id        7992 non-null   object
 1   title           7992 non-null   object
 2   channel_title   7992 non-null   object
 3   category_id     7992 non-null   int64 
 4   tags            7992 non-null   object
 5   views           7992 non-null   int64 
 6   likes           7992 non-null   int64 
 7   dislikes        7992 non-null   int64 
 8   comment_total   7992 non-null   int64 
 9   thumbnail_link  7992 non-null   object
 10  date            7992 non-null   object
dtypes: int64(5), object(6)
memory usage: 686.9+ KB


In [21]:
from datetime import datetime

df_youtube['date'] = df_youtube['date'] + '.2020'
df_youtube['date'] = pd.to_datetime(df_youtube['date'].astype(str), format='%d.%m.%Y')
df_youtube.head()

,video_id,title,channel_title,category_id,tags,views,likes,dislikes,comment_total,thumbnail_link,date
0,XpVt6Z1Gjjo,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,Logan Paul Vlogs,24,logan paul vlog|logan paul|logan|paul|olympics...,4394029,320053,5931,46245,https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg,2020-09-13
1,K4wEI5zhHB0,iPhone X — Introducing iPhone X — Apple,Apple,28,Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lig...,7860119,185853,26679,0,https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg,2020-09-13
2,cLdxuaxaQwc,My Response,PewDiePie,22,[none],5845909,576597,39774,170708,https://i.ytimg.com/vi/cLdxuaxaQwc/default.jpg,2020-09-13
3,WYYvHb03Eog,Apple iPhone X first look,The Verge,28,apple iphone x hands on|Apple iPhone X|iPhone ...,2642103,24975,4542,12829,https://i.ytimg.com/vi/WYYvHb03Eog/default.jpg,2020-09-13
4,sjlHnJvXdQs,iPhone X (parody),jacksfilms,23,jacksfilms|parody|parodies|iphone|iphone x|iph...,1168130,96666,568,6666,https://i.ytimg.com/vi/sjlHnJvXdQs/default.jpg,2020-09-13


### Importer les données

In [22]:
import json
with open('./data/US_category_id.json', "r") as json_file:
    json_data = json.load(json_file)
    #collection.insert_one(json_data)
    for data in json_data['items']: 
        data['channelId']=data['snippet']['channelId']
        data['title']=data['snippet']['title']
        data['assignable']=data['snippet']['assignable']
        data.pop('snippet')
    df_json=pd.DataFrame.from_dict(json_data['items'])
    df_json['category_id']=df_json['id'].astype(int)
    df_json.drop('id', axis='columns', inplace=True)
df_json.head()

,kind,etag,channelId,title,assignable,category_id
0,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/Xy1mB4_yLrHy_BmKm...",UCBR8-60-B28hp2BmDPdntcQ,Film & Animation,True,1
1,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/UZ1oLIIz2dxIhO45Z...",UCBR8-60-B28hp2BmDPdntcQ,Autos & Vehicles,True,2
2,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/nqRIq97-xe5XRZTxb...",UCBR8-60-B28hp2BmDPdntcQ,Music,True,10
3,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/HwXKamM1Q20q9BN-o...",UCBR8-60-B28hp2BmDPdntcQ,Pets & Animals,True,15
4,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/9GQMSRjrZdHeb1OEM...",UCBR8-60-B28hp2BmDPdntcQ,Sports,True,17


In [23]:
df=pd.merge(df_youtube,df_json,on='category_id')

In [24]:
df.head()

,video_id,title_x,channel_title,category_id,tags,views,likes,dislikes,comment_total,thumbnail_link,date,kind,etag,channelId,title_y,assignable
0,XpVt6Z1Gjjo,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,Logan Paul Vlogs,24,logan paul vlog|logan paul|logan|paul|olympics...,4394029,320053,5931,46245,https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg,2020-09-13,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/UVB9oxX2Bvqa_w_y3...",UCBR8-60-B28hp2BmDPdntcQ,Entertainment,True
1,4MkC65emkG4,Hand In Hand A Benefit For Hurricane Relief | MTV,MTV,24,mtv|video|online|official|tv|television|watch|...,274358,9215,477,838,https://i.ytimg.com/vi/4MkC65emkG4/default.jpg,2020-09-13,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/UVB9oxX2Bvqa_w_y3...",UCBR8-60-B28hp2BmDPdntcQ,Entertainment,True
2,vu_9muoxT50,Colin Cloud: Mind Reader Predicts Your Tweets ...,America's Got Talent,24,America's Got Talent 2017|america's got talent...,473691,14740,415,1696,https://i.ytimg.com/vi/vu_9muoxT50/default.jpg,2020-09-13,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/UVB9oxX2Bvqa_w_y3...",UCBR8-60-B28hp2BmDPdntcQ,Entertainment,True
3,ZQK1F0wz6z4,What Do You Want to Eat?!,Wong Fu Productions,24,panda|what should we eat|buzzfeed|comedy|boyfr...,282858,14870,300,1398,https://i.ytimg.com/vi/ZQK1F0wz6z4/default.jpg,2020-09-13,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/UVB9oxX2Bvqa_w_y3...",UCBR8-60-B28hp2BmDPdntcQ,Entertainment,True
4,5ywKal6-anc,Gigi Hadid Loses High Heel During Fashion Week...,TMZ,24,TMZ2016FS11221|TMZ|Hollywood|Celebrity|Enterta...,703750,2921,2196,1042,https://i.ytimg.com/vi/5ywKal6-anc/default.jpg,2020-09-13,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/UVB9oxX2Bvqa_w_y3...",UCBR8-60-B28hp2BmDPdntcQ,Entertainment,True


In [25]:
collection.insert_many(df.to_dict('records'))
collection.find_one()

{'_id': ObjectId('5fce51d61a6ced4cfcb015c1'),
 'video_id': 'XpVt6Z1Gjjo',
 'title_x': '1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED YOUTUBE FOREVER!',
 'channel_title': 'Logan Paul Vlogs',
 'category_id': 24,
 'tags': 'logan paul vlog|logan paul|logan|paul|olympics|logan paul youtube|vlog|daily|comedy|hollywood|parrot|maverick|bird|maverick clothes|diamond play button|logan paul diamond play button|10M subscribers|logan paul 1 year vlogging|1 year vlog|dwarf mamba play button|logan paul history|youtube history|10M|10M plaque|youtube button|diamond button|logang|logang 4 life',
 'views': 4394029,
 'likes': 320053,
 'dislikes': 5931,
 'comment_total': 46245,
 'thumbnail_link': 'https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg',
 'date': datetime.datetime(2020, 9, 13, 0, 0),
 'kind': 'youtube#videoCategory',
 'etag': '"m2yskBQFythfE4irbTIeOgYYfBU/UVB9oxX2Bvqa_w_y3vXSLVK5E_s"',
 'channelId': 'UCBR8-60-B28hp2BmDPdntcQ',
 'title_y': 'Entertainment',
 'assignable': True}

## Question 1  

In [26]:
apple=collection.find({'channel_title':'Apple'})
next(apple)

{'_id': ObjectId('5fce51d61a6ced4cfcb01c02'),
 'video_id': 'K4wEI5zhHB0',
 'title_x': 'iPhone X — Introducing iPhone X — Apple',
 'channel_title': 'Apple',
 'category_id': 28,
 'tags': 'Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lighting|A11 Bionic|augmented reality|emoji|animoji|Face ID|Apple Pay|camera|smartphone',
 'views': 7860119,
 'likes': 185853,
 'dislikes': 26679,
 'comment_total': 0,
 'thumbnail_link': 'https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg',
 'date': datetime.datetime(2020, 9, 13, 0, 0),
 'kind': 'youtube#videoCategory',
 'etag': '"m2yskBQFythfE4irbTIeOgYYfBU/w5HjcTD82G_XA3xBctS30zS-JpQ"',
 'channelId': 'UCBR8-60-B28hp2BmDPdntcQ',
 'title_y': 'Science & Technology',
 'assignable': True}

## Question 2

In [27]:
count = collection.aggregate([{"$group" : {"_id" : "$category_id", "number" : {"$sum" : 1}}}])
list(count)

[{'_id': 26, 'number': 869},
 {'_id': 22, 'number': 882},
 {'_id': 43, 'number': 2},
 {'_id': 2, 'number': 116},
 {'_id': 25, 'number': 623},
 {'_id': 28, 'number': 512},
 {'_id': 24, 'number': 1601},
 {'_id': 10, 'number': 1250},
 {'_id': 15, 'number': 116},
 {'_id': 27, 'number': 334},
 {'_id': 17, 'number': 410},
 {'_id': 20, 'number': 82},
 {'_id': 23, 'number': 755},
 {'_id': 1, 'number': 378},
 {'_id': 29, 'number': 14},
 {'_id': 19, 'number': 48}]

## Question 3

In [28]:
all=collection.find()
for doc in all:
    collection.update_one(doc,{'$set':{'tags':doc['tags'].split('|')}})

In [29]:
next(collection.find())

{'_id': ObjectId('5fce51d61a6ced4cfcb015c1'),
 'video_id': 'XpVt6Z1Gjjo',
 'title_x': '1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED YOUTUBE FOREVER!',
 'channel_title': 'Logan Paul Vlogs',
 'category_id': 24,
 'tags': ['logan paul vlog',
  'logan paul',
  'logan',
  'paul',
  'olympics',
  'logan paul youtube',
  'vlog',
  'daily',
  'comedy',
  'hollywood',
  'parrot',
  'maverick',
  'bird',
  'maverick clothes',
  'diamond play button',
  'logan paul diamond play button',
  '10M subscribers',
  'logan paul 1 year vlogging',
  '1 year vlog',
  'dwarf mamba play button',
  'logan paul history',
  'youtube history',
  '10M',
  '10M plaque',
  'youtube button',
  'diamond button',
  'logang',
  'logang 4 life'],
 'views': 4394029,
 'likes': 320053,
 'dislikes': 5931,
 'comment_total': 46245,
 'thumbnail_link': 'https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg',
 'date': datetime.datetime(2020, 9, 13, 0, 0),
 'kind': 'youtube#videoCategory',
 'etag': '"m2yskBQFythfE4irbTIeOgYYfBU/UVB9oxX2B

## Question 4

In [30]:
plus_vue=collection.find().sort([('views',-1)]).limit(1)
next(plus_vue)

{'_id': ObjectId('5fce51d61a6ced4cfcb02a83'),
 'video_id': 'MBdVXkSdhwU',
 'title_x': "BTS (방탄소년단) 'DNA' Official MV",
 'channel_title': 'ibighit',
 'category_id': 10,
 'tags': ['BIGHIT', '빅히트', '방탄소년단', 'BTS', 'BANGTAN', '방탄'],
 'views': 41500672,
 'likes': 2010366,
 'dislikes': 78076,
 'comment_total': 736179,
 'thumbnail_link': 'https://i.ytimg.com/vi/MBdVXkSdhwU/default.jpg',
 'date': datetime.datetime(2020, 9, 22, 0, 0),
 'kind': 'youtube#videoCategory',
 'etag': '"m2yskBQFythfE4irbTIeOgYYfBU/nqRIq97-xe5XRZTxbknKFVe5Lmg"',
 'channelId': 'UCBR8-60-B28hp2BmDPdntcQ',
 'title_y': 'Music',
 'assignable': True}

## Question 5

In [31]:
moyen=collection.aggregate([{'$group':{'_id': '$category_id', 'moyen_view':{'$avg' : '$views'}}}])
list(moyen)

[{'_id': 28, 'moyen_view': 924730.548828125},
 {'_id': 20, 'moyen_view': 681081.8170731707},
 {'_id': 17, 'moyen_view': 728434.3243902439},
 {'_id': 27, 'moyen_view': 547582.4700598803},
 {'_id': 1, 'moyen_view': 1039472.6560846561},
 {'_id': 29, 'moyen_view': 1110334.2142857143},
 {'_id': 19, 'moyen_view': 464041.0833333333},
 {'_id': 26, 'moyen_view': 537665.1806674339},
 {'_id': 43, 'moyen_view': 8492.5},
 {'_id': 23, 'moyen_view': 1240073.294039735},
 {'_id': 2, 'moyen_view': 607693.8706896552},
 {'_id': 25, 'moyen_view': 540955.5569823436},
 {'_id': 22, 'moyen_view': 971532.7573696146},
 {'_id': 24, 'moyen_view': 1154868.4397251718},
 {'_id': 10, 'moyen_view': 1176553.612},
 {'_id': 15, 'moyen_view': 651404.3879310344}]

## Question 6 

In [32]:
moyen_like=collection.aggregate([
    {'$group':{'_id':'$channel_title', 'moyen_likes':{'$avg':'$likes'}}},
    {'$sort':{'moyen_likes':-1}}])

next(moyen_like)

{'_id': 'ZaynVEVO', 'moyen_likes': 1431683.0}